In [ ]:
import pandas as pd
import imageio
import glob
import matplotlib
import matplotlib.pyplot as plt
import os
import cv2
from copy import deepcopy
import json
import numpy as np
from skimage.transform import resize
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense,Activation,Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras import backend as K
import keras
import tensorflow as tf

def normalization_image(image):
    """
    This function reads a numpy array of 
    images and return the contents as a 
    numpy array.
    
    Parameters
    ----------
    image : numpy array
        It has images as numpy array.
        
    Returns
    ----------
    images: numpy array
        Normalized the value of array
        between 0 to 1.
    """
    
    image= image.astype('float32')
    image= image - np.mean(image)
    image = image / np.std(image)
    image = image[...,np.newaxis]
    return image
    
def normalization_mask(mask):
    """
    This function reads a numpy array of 
    mask and return the normalized contents
    as a numpy array.
    
    Parameters
    ----------
    mask : numpy array
        It has mask as numpy array.

    Returns
    ----------
    mask: numpy array
        Normalized the value of array
        between 0 to 1.
    """
    
    mask = mask/255
    mask=mask[...,np.newaxis]
    return mask


def mask_preprocessing(mask_dir_path):
    """
    This function reads a path of the
    mask and return the gray scaled image 
    as a numpy array.
    
    Parameters
    ----------
    mask_dir_path : str
        Read the mask directory.

    Returns
    ----------
    gray_img: numpy array
        Return gray scale image as numpy array
    """
    mask_files = [f for f in glob.glob(mask_dir_path + "**/*color.png", recursive=True)]
    print("number of mask file :::",len(mask_files))
    mask_files.sort()
    gray_img=[]
    for i in mask_files:
        mask_arr=imageio.imread(i)
        gray_arr=cv2.cvtColor(mask_arr, cv2.COLOR_BGR2GRAY)
        gray_img.append(gray_arr)
        gray_img.append(resize(gray_arr,(224,224),preserve_range=True))
    gray_img=np.array(gray_img)
    return gray_img
    
def image_preprocessing(image_dir_path):
    """
    This function reads a path of the
    image and return the gray scaled 
    image as a numpy array.
    
    Parameters
    ----------
    image_dir_path : str
        Read the mask directory.

    Returns
    ----------
    image_array: numpy array
        Return gray scale image as numpy array
    """
    images_files = [f for f in glob.glob(image_dir_path + "**/*.png", recursive=True)]
    print("number of mask file :::",len(images_files))
    images_files.sort()
    image_array=[]
    for i in images_files:
        img=imageio.imread(i)
        img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        image_array.append(resize(img,(224,224),preserve_range=True))
    image_array=np.array(image_array)
    return image_array

def unet_model(output_channels):
    
    """
    This function create the unet architecture
    using mobile_net as a down sample layer
    and transposed convultion layer as the upsample 
    layer.
    
    Parameters
    ----------
    output_channels : integer
        number of output channel in train mask

    Returns
    ----------
        Return keras model object
    """

  # This is the last layer of the model
  last = keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same', activation='softmax')  #64x64 -> 128x128

  inputs = keras.layers.Input(shape=[224, 224, 3])
  x = inputs

  # Downsampling through the model
  skips = down_stack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = keras.layers.Concatenate()
    x = concat([x, skip])

  x = last(x)

  return keras.Model(inputs=inputs, outputs=x)

def upsample(filters, size, apply_dropout=False):
    """
    This function create the upsampling 
    layer for the model using transposed
    convolution
    
    Parameters
    ----------
    filter : integer
        Take value of filter used for upsampling
    size : integer
        take size of the mask

    Returns
    ----------
    result : kearas layer
        Return upsampling layer.
    """
    
  initializer = tf.random_normal_initializer(0., 0.02)

  result = keras.Sequential()
  result.add(
    keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(keras.layers.Dropout(0.5))

  result.add(keras.layers.ReLU())

  return result

def prediction(testing_image_arr):
    """
    This function predit the mask of the test
    image. Return the predicted mask array
    
    Parameters
    ----------
    testing_image_arr : numpy array
        test image numpy array.
        
    Returns
    ----------
    mask_pred : array
        predicted mask arry.
    """
    
    mask_pred=[]
    for image in testing_image_arr:
        image=image[np.newaxis,...]
        predicted = model.predict(image, verbose=1)
        mask_pred.append(predicted)
    return mask_pred
        
def calculating_iou(mask_real,temp_mask):
    """
    This function calculate the intersection
    over union.
    
    Parameters
    ----------
    mask_real : array
        ground truth array.
    temp_mask : array
        predicted mask array.
        
    Returns
    ----------
    mean_iou : decimal
        mean intersection over union.
    """
    arr_iou=[]
    for i in range(len(mask_real)):
        intersection = np.logical_and(mask_real[i],temp_mask[i])
        union = np.logical_or(mask_real[i], temp_mask[i])
        iou_score = np.sum(intersection) / np.sum(union)
        arr_iou.append(iou_score)
    arr_iou=np.array(arr_iou)
    arr_iou= np.nan_to_num(arr_iou, copy=True, nan=1.0)
    mean_iou=arr_iou.mean()
    return mean_iou



In [ ]:
if __name__ == '__main__':
    
    parser = argparse.ArgumentParser(
        description=('Trains the model and outputs predictions.'),
        add_help='How to use', prog='model.py <args>')

    # Required arguments path of dataset folder "home/jupyter/dataset/"
    parser.add_argument("-d", "--data_path", required=True,
                        help=("Provide the path to the data folder"))

    args = vars(parser.parse_args())
    
    #passing the path of image and mask file to be trained and tested
    mask_dir_path = read_file(args['data_path'] + '/mask/train/')
    image_dir_path = read_file(args['data_path'] + '/image/train')
    test_image_dir_path = read_file(args['data_path'] + '/image/val/')
    ground_truth = read_file(args['data_path'] + '/mask/val/')
    
    #numpy array of the mask after normalization
    train_mask=mask_preprocessing(mask_dir_path)
    train_mask=loading_json(mask_dir_path,train_mask)
    train_mask=normalization_mask(train_mask)
    
    #numpy array of the image after normalization
    train_image=image_preprocessing(image_dir_path)
    train_image=normalization_image(train_image)
    
    #numpy array of the test mask after normalization
    test_mask=mask_preprocessing(ground_truth)
    test_mask=loading_json(ground_truth,train_mask)
    test_mask=normalization_mask(test_mask)
    
    #numpy array of the test image after normalization
    test_image=image_preprocessing(test_image_dir_path)
    test_image=normalization_image(test_image)
    
    
    # setting up the model using mobile_net_v2
    base_model = MobileNetV2(include_top=False)
    # Use the activations of these layers
    layer_names = [
        'block_1_expand_relu',   # 64x64
        'block_3_expand_relu',   # 32x32
        'block_6_expand_relu',   # 16x16
        'block_13_expand_relu',  # 8x8
        'block_16_project',      # 4x4
    ]
    #removing the top layer of the mobile_net_v2
    layers = [base_model.get_layer(name).output for name in layer_names]
    #downsampling layer the semantic segementation model using Mobile_net_v2
    down_stack = Model(inputs=base_model.input, outputs=layers)
    # storing the upsample layer for concatenation
    up_stack = [
    upsample(512, 3),  # 4x4 -> 8x8
    upsample(256, 3),  # 8x8 -> 16x16
    upsample(128, 3),  # 16x16 -> 32x32
    upsample(64, 3),   # 32x32 -> 64x64
    ]
    # calling model
    model=unet_model(3)
    # compiling the model.
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    # fitting the model
    model.fit(train_image, train_mask, batch_size=1, epochs=5, verbose=1, shuffle=True)
    # predicted mask
    prediction_mask=prediction(test_image)
    prediction_mask=np.array(prediction_mask)
    #calculating intersection over union
    mean_iou=calculating_iou(ground_truth,prediction_mask)
    print("mean_iou:::",mean_iou)